In [ ]:
# Running this on an ESP8266 of type NodeMCU

# For ESP32 see below

#CLK/SCK <-> SCK (14) <-> D5
#DO/MISO <-> MISO (12) <-> D6
#DI/MOSI <-> MOSI (13) <-> D7
#CS <-> CS (15) <-> D8


In [12]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [2]:
%sendtofile --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py

Sent 272 lines (8218 bytes) to sdcard.py.


In [13]:
import machine
help(machine.SPI)

object <class 'SoftSPI'> is of type type
  init -- <function>
  deinit -- <function>
  read -- <function>
  readinto -- <function>
  write -- <function>
  write_readinto -- <function>
  MSB -- 0
  LSB -- 1


In [14]:
import machine, sdcard, os
k = machine.SPI(1)
sd = sdcard.SDCard(k, machine.Pin(15))
os.mount(sd, '/sd')

In [18]:
help(k)

object SPI(id=1, baudrate=100000, polarity=0, phase=0, bits=8, firstbit=0, sck=-1, mosi=-1, miso=-1) is of type SPI
  init -- <function>
  deinit -- <function>
  read -- <function>
  readinto -- <function>
  write -- <function>
  write_readinto -- <function>
  MSB -- 0
  LSB -- 1


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "sdcard.py", line 54, in __init__
  File "sdcard.py", line 82, in init_card
OSError: no SD card


In [33]:
# direct call in a loop has stalls in it
import time
n = 3000
fout = open("/sd/data15.txt", "w")
for i in range(n):
    t = time.ticks_ms()
    fout.write("%08d" % t)
fout.close()


.

In [34]:
%fetchfile --binary "/sd/data15.txt"

Fetched 24000=24000 bytes from /sd/data15.txt.
Saving file to 'data15.txt'

In [35]:
# first the ring buffer
import array, time
nt = 20
pt = array.array("I", range(0,nt+2))
 # [i, tprev, dt, dt, ..., dt]
pt[1] = time.ticks_ms()
pt[0] = 0
def irqtrigger(p):
    t1 = time.ticks_ms()
    i = pt[0]+1
    pt[(i%nt)+2] = t1 # - pt[1]
    pt[1] = t1
    pt[0] = i


In [36]:
# Set the interrupt timer
from machine import Timer
timer = Timer(0)
timer.init(period=10, mode=Timer.PERIODIC, callback=irqtrigger)


In [41]:
# Fetch data off the buffer as they arrive and save to file
fname = "/sd/data16.txt"  
fname = "data17.txt" # for internal flash
import time
n = 3000
fout = open(fname, "w")
j = pt[0]
for i in range(n):
    while j == pt[0]:
        time.sleep_ms(2)
    while j != pt[0]:
        j += 1
        fout.write("%08d" % pt[(j%nt)+2])
fout.close()


......

In [38]:
%fetchfile --binary "/sd/data16.txt"

Fetched 25112=25112 bytes from /sd/data16.txt.
Saving file to 'data16.txt'

In [43]:
%fetchfile --binary "data17.txt"

Fetched 24800=24800 bytes from data17.txt.
Saving file to 'data17.txt'

Plot this using code on the PC notebook

```
from matplotlib import pyplot as plt
%matplotlib inline

k = open("data15.txt").read()
k = [ int(k[i-8:i])  for i in range(8, len(k), 8) ]
k = [a-b  for a,b in zip(k[1:], k)]
plt.plot(k)
```

In [30]:
%fetchfile --binary "/sd/data16.txt"

Fetched 24976=24976 bytes from /sd/data16.txt.
Saving file to 'data16.txt'

In [21]:
# Try the SD card on the ESP32
%serialconnect

[leftinbuffer] ['ets Jun  8 2016 00:22:57']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:4436']
[leftinbuffer] ['load:0x40078000,len:0']
[leftinbuffer] ['load:0x40078000,len:11816']
[leftinbuffer] ['entry 0x4007a9fc']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "main.py", line 2, in <module>']
[leftinbuffer] ['  File "OLED_driver.py", line 5, in <module>']
[leftinbuffer] ['AssertionError: OLED i2c not found']
[leftinbuffer] ['MicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32']
[leftinbuffer] ['Type "help()" for more information.']
[leftinbuffer] ['>>> ']
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [66]:
%sendtofile --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py

Sent 272 lines (8218 bytes) to sdcard.py.


In [55]:
from machine import Pin, SPI
spi = SPI(sck=Pin(5), mosi=Pin(22), miso=Pin(27))  # should be mosi=Pin19 but is broke on screenless heltec

In [51]:
p = machine.Pin(19, machine.Pin.OUT)
p.value(1)
print(p.value())

0


In [56]:
sd = sdcard.SDCard(spi, machine.Pin(23))


In [57]:
import machine, sdcard, os
os.mount(sd, '/sd')

In [60]:
print(os.listdir("/sd/OOU"))

['000.TXT', '001.TXT', '002.TXT', '003.TXT', '004.TXT', '005.TXT', '006.TXT', '007.TXT', '008.TXT', '009.TXT', '010.TXT', '011.TXT', '012.TXT', '013.TXT', '014.TXT', '015.TXT', '016.TXT', '017.TXT', '018.TXT', '019.TXT', '020.TXT', '021.TXT', '022.TXT', '023.TXT', '024.TXT', '025.TXT', '026.TXT', '027.TXT', '028.TXT', '029.TXT', '030.TXT', '031.TXT', '032.TXT']


In [61]:
%fetchfile --binary "/sd/OOU/032.txt"

Fetched 1226518=1226518 bytes from /sd/OOU/032.txt.
Saving file to '032.txt'

In [77]:
# direct call in a loop has stalls in it
import time
n = 3000
fout = open("/sd/data25.txt", "w")
for i in range(n):
    t = time.ticks_ms()
    fout.write("%08d" % t)
fout.close()


.

In [78]:
%fetchfile --binary "/sd/data25.txt"

Fetched 24000=24000 bytes from /sd/data25.txt.
Saving file to 'data25.txt'

In [79]:
# first the ring buffer
import array, time
nt = 20
pt = array.array("I", range(0,nt+2))
 # [i, tprev, dt, dt, ..., dt]
pt[1] = time.ticks_ms()
pt[0] = 0
def irqtrigger(p):
    t1 = time.ticks_ms()
    i = pt[0]+1
    pt[(i%nt)+2] = t1 # - pt[1]
    pt[1] = t1
    pt[0] = i


In [80]:
# Set the interrupt timer
from machine import Timer
timer = Timer(0)
timer.init(period=10, mode=Timer.PERIODIC, callback=irqtrigger)


In [83]:
# Fetch data off the buffer as they arrive and save to file
fname = "/sd/data26.txt"  
fname = "data27.txt" # for internal flash
import time
n = 3000
fout = open(fname, "w")
j = pt[0]
for i in range(n):
    while j == pt[0]:
        time.sleep_ms(2)
    while j != pt[0]:
        j += 1
        fout.write("%08d" % pt[(j%nt)+2])
fout.close()


......

In [82]:
%fetchfile --binary "/sd/data26.txt"

Fetched 24368=24368 bytes from /sd/data26.txt.
Saving file to 'data26.txt'

In [84]:
%fetchfile --binary "data27.txt"

Fetched 24312=24312 bytes from data27.txt.
Saving file to 'data27.txt'